In [44]:
import pandas as pd
import pickle
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score
from statsmodels.tsa.vector_ar.var_model import VAR

In [49]:
csv_dataset = '../data/processed/data.csv'
csv = pd.read_csv(csv_dataset, encoding='utf_8')
df = pd.DataFrame(csv)
df = df.drop(columns='date')

In [50]:
# Set the lags to 72
lags = 72

# Split the data into training and testing sets
train = df[:-lags]
test = df[-lags:]

# Fit a VAR model with 72 lags
model = VAR(train)
results = model.fit(lags)

# Make predictions for the next 3 days
predictions = results.forecast(test.values, steps=lags)

# Convert the predictions back to a DataFrame
predictions = pd.DataFrame(predictions, columns=df.columns, index=test.index)

# Calculate MSE and MAE for the test data
mse_test = mean_squared_error(test, predictions)
mae_test = mean_absolute_error(test, predictions)
evs_test = explained_variance_score(test, predictions)

# Make predictions for the training data
predictions_train = results.fittedvalues

train = train[lags:]
# Calculate MSE and MAE for the training data
mse_train = mean_squared_error(train, predictions_train)
mae_train = mean_absolute_error(train, predictions_train)
evs_train = explained_variance_score(train, predictions_train)

# Print the results
print('MSE train:', mse_train)
print('MAE train:', mae_train)
print('MSE test:', mse_test)
print('MAE test:', mae_test)

MSE train: 0.764211575621414
MAE train: 0.4612037826366892
MSE test: 449.89493849715575
MAE test: 12.913437839777213


In [51]:
with open('../reports/train_metrics.txt', 'w') as file:
    file.write('MAE:' + str(mae_train) + '\n')
    file.write('MSE:' + str(mse_train) + '\n')
    file.write('EVS:' + str(evs_train) + '\n')

In [52]:
with open('../reports/metrics.txt', 'w') as file:
    file.write('MAE:' + str(mae_test) + '\n')
    file.write('MSE:' + str(mse_test) + '\n')
    file.write('EVS:' + str(evs_test) + '\n')


In [54]:
with open('../models/var_model', 'wb') as f:
    pickle.dump(results, f)